In [1]:
# !pip install keras
# !pip install tensorflow
!pip install xgboost

     |████████████████████████████████| 166.7 MB 12 kB/s /s eta 0:00:01


In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")

In [3]:
xls = pd.ExcelFile('2 years data.xlsx',engine='openpyxl')
df1 = pd.read_excel(xls, 'Planilha1')
df2 = pd.read_excel(xls, 'Price increase dates')
df3 = pd.read_excel(xls, 'Mandatory vacation dates',names=['start','end'])

df1 = df1.iloc[:245815]
df2.dropna(axis=1,inplace=True)
df3.dropna(inplace=True)
df3.drop([0],inplace=True)
df1.drop(['Order nº','Custumer Name','Item Name','Subgroup_description'],axis=1,inplace=True)
df2.drop(['Subgroup of products (description)'],axis=1,inplace=True)

le = preprocessing.LabelEncoder()
for col in ['Country','State','City']:
    df1[col] = le.fit_transform(df1[[col]])
df = df1.groupby(['Order date','Custumer Id','Item Id','Country','State','City','Subgruop_ID']).mean().reset_index()

## recency, frequency, price_increase_30, next_buy
### is_vacation, vacation_30

In [4]:
def time_series_bin(right,col):

    ret = pd.DataFrame(pd.date_range('2019-10-01','2021-09-13',freq='d'),columns=['Date'])
    try:
        new_data = pd.merge(ret,right,left_on='Date',right_on=col,how='outer')
    except:
        new_data = pd.concat(ret,right,axis=1,join="outer",)
    new_data['ret']= new_data[col].isna()==0
    new_data['ret'].replace({True:1,False:0},inplace=True)
    return new_data['ret']

In [5]:
def recency(dfr,cusid):

    t = time_series_bin(dfr.loc[dfr['Custumer Id']==cusid][['Order date']].drop_duplicates(),'Order date')
    t = np.where(t.to_numpy()==1)[0]
    t1 = [t[i]-t[i+1] for i in range(len(t)-1)]
    tt = [0-t[0]]
    [tt.append(i) for i in t1]
    tt.append(t[-1]-714)

    data = []
    for rec in tt :
        [data.append(_) for _ in range(abs(rec))]

    return data

In [6]:
def buy_freq(dfr,cusid):
    t = time_series_bin(dfr.loc[dfr['Custumer Id']==cusid][['Order date']].drop_duplicates(),'Order date')
    ret = [0]
    [ret.append(t[:i+1].mean()) for i in range(len(t)-1)]
    
    return ret

In [7]:
def price_inc_30(df2):
    df2 = pd.read_excel(xls, 'Price increase dates')
    df2.dropna(axis=1,inplace=True)
    df2.drop(['Subgroup of products (description)'],axis=1,inplace=True)
    df2 = pd.concat([df2[['Date of price increase']]
                     ,pd.DataFrame.sparse.from_spmatrix((OneHotEncoder().fit_transform(df2.drop(['Date of price increase']
                                                                                                ,axis=1))))
                    ]
                    ,axis=1)
    df2 = df2.groupby(['Date of price increase']).sum()

    x = pd.DataFrame(index=pd.date_range('2019-10-01','2021-09-13',freq='d'))
    x = pd.concat([x,df2],join="outer",axis=1,)
    x.fillna(0,inplace=True)
    return x.reset_index().drop(['index'],axis=1)

In [8]:
def next_buy(dfr,cusid):
    t = time_series_bin(dfr.loc[dfr['Custumer Id']==cusid][['Order date']].drop_duplicates(),'Order date')
    t = np.where(t.to_numpy()==1)[0]
    t1 = [t[i+1]-t[i] for i in range(len(t)-1)]
    t2 = []
    tt = [t[0]]
    [tt.append(i) for i in t1]
    tt.append(713-t[-1])
    
    for i in range(len(tt)-1):
        for j in range(tt[i]):
            t2.append(tt[i]-j-1)
    
    for i in range(714-len(t2)):
        t2.append(90)
    return(t2)

In [9]:
def data_cus(df1,df2,cusid):
    ret = pd.DataFrame()
    ret['recency'] = recency(df1,cusid)
    ret['buy_freq'] = buy_freq(df1,cusid)
    for col in price_inc_30(df2).columns:
        ret['price_inc_'+str(col)] = price_inc_30(df2)[col]
    ret['next_buy'] = next_buy(df1,cusid)
    return ret

In [10]:
scale = StandardScaler()
data = scale.fit_transform(data_cus(df,df2,750))

In [11]:
train, test = data[:600], data[600:]
x_train, x_test, y_train, y_test = train[:,:-1], test[:,:-1], train[:,-1], test[:,-1]

In [12]:
model = SVR().fit(x_train, y_train)
model.score(x_test, y_test)

0.6321132533994073

In [13]:
mean_squared_error(model.predict(x_test),y_test)

0.4857792889181487

In [ ]:
param_grid = {
    'C': [0.001, 0.01, 0.1],
    'gamma': [10, 1, 0.1, 0.01],
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'degree':[2, 3, 5],  
}
grid = GridSearchCV(SVR(), param_grid, refit=True, verbose=5, cv=5, scoring=mean_squared_error, n_jobs=10)
grid.fit(x_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


In [ ]:
print(grid.best_estimator_)

In [ ]:
grid_predictions = grid.predict(x_test)

In [ ]:
mean_squared_error(grid_predictions,y_test)

In [ ]:
grid.score(x_test,y_test)